In [33]:
import numpy as np
import cv2
import glob
import scipy.optimize
from cv2 import aruco
from marker_tracker import markersOnPen, arucoParams

def marker_jacobian(dUVdYXZ, dUVdT, marker_count, img_count, marker_id, img_id):
    # Rows: [u1, v1, u2, v2, u3, v3, u4, v4] for single image
    jac = np.zeros((8, 3*4*marker_count + 6*img_count), dtype=np.float32)
    marker_col_start = marker_id * 4 * 3
    for i in range(4):
        jac[i*2:i*2+2, marker_col_start + i*3:i*3+3+marker_col_start] = dUVdYXZ

    camera_col_start = 4 * 3 * marker_count + 6 * img_id
    jac[:, camera_col_start:camera_col_start+6] = dUVdT
    return jac


def residual(x: np.ndarray, x0: np.ndarray, camera_matrix, dist_coeffs, images: list[dict[int, np.ndarray]]):
    marker_count = 8
    marker_poses = x[0:marker_count*4*3].reshape((marker_count, 4, 3))
    camera_poses = x[marker_count*4*3:].reshape((-1, 6))
    res_all = []
    jac_all = []
    for img_id in range(len(images)):
        img = images[img_id]
        rvec = camera_poses[img_id, 0:3]
        tvec = camera_poses[img_id, 3:6]

        T = camera_matrix @ np.hstack((cv2.Rodrigues(rvec)[0], tvec.reshape((3,1))))
        dUVdXYZ = T[0:2, 0:3] # Jacobian of the projection w.r.t. the marker pose, for a single point
        for marker_id, marker_corners_observed in img.items():
            canonical_marker_id = marker_id  # - 92
            projected: np.ndarray
            jac: np.ndarray
            projected, jac = cv2.projectPoints(
                objectPoints=marker_poses[canonical_marker_id,:,:], # 4x3
                rvec=rvec,
                tvec=tvec,
                cameraMatrix=camera_matrix,
                distCoeffs=dist_coeffs,
            )

            dUVdT = jac[:,0:6] # Jacobian of the projection w.r.t. the camera pose, for all points

            marker_jac = marker_jacobian(dUVdXYZ, dUVdT, marker_count, len(images), canonical_marker_id, img_id)

            res = projected.flatten() - marker_corners_observed.flatten()
            res_all.append(res)
            jac_all.append(marker_jac)
    # Also minimise difference from initial estimates:
    change_min_scaling = 1000
    res_all.append((x[0:marker_count*4*3] - x0[0:marker_count*4*3]) * change_min_scaling)
    jac_all.append(np.hstack((
        np.eye(marker_count*4*3),
        np.zeros((marker_count*4*3, 6*len(images))),
    )) * change_min_scaling)
    # Each column of jac_all is one input variable
    # Each row is one residual
    return (np.concatenate(res_all), np.vstack(jac_all))

def get_observed_points(pathname, arucoParams):
    observed_points = []
    for iname in glob.glob(pathname):
        img = cv2.imread(iname)
        # TODO: Better params
        corners_all, ids, _ = aruco.detectMarkers(
            image=img,
            dictionary=aruco.getPredefinedDictionary(aruco.DICT_4X4_100),
            parameters=arucoParams,
        )
        corner_dict = {}
        for i in range(ids.shape[0]):
            corner_dict[ids[i, 0] - 92] = corners_all[i][0,:,:]
        observed_points.append(corner_dict)
    return observed_points

def get_initial_estimate(observed_points: list[dict[int, np.ndarray]], camera_matrix, dist_coeffs):
    marker_count = 8
    marker_poses = np.zeros((marker_count, 4, 3), dtype=np.float32)
    for mid, corners in markersOnPen.items():
        marker_poses[mid - 92,:,:] = corners

    camera_poses = np.zeros((len(observed_points), 6), dtype=np.float32)
    for img_id, img in enumerate(observed_points):
        validMarkers = []
        for marker_id, marker_corners_observed in img.items():
            if marker_id + 92 in markersOnPen:
                validMarkers.append((markersOnPen[marker_id+92], marker_corners_observed))

        screenCorners = np.concatenate([cornersIS for _, cornersIS in validMarkers])
        penCorners = np.concatenate([cornersPS for cornersPS, _ in validMarkers])
        success, rvec, tvec = cv2.solvePnP(
            penCorners,
            screenCorners,
            camera_matrix,
            dist_coeffs,
            flags=cv2.SOLVEPNP_SQPNP,
        )
        # rvec = np.zeros((3,), dtype=np.float32)
        # tvec = np.zeros((3,), dtype=np.float32)
        camera_poses[img_id, :] = np.hstack((rvec.flatten(), tvec.flatten()))
    # marker_count = 8
    # marker_poses = np.zeros((marker_count, 4, 3), dtype=np.float32)
    return np.concatenate((marker_poses.flatten(), camera_poses.flatten()))

def calibrate_markers(camera_matrix, dist_coeffs, images: list[dict[int, np.ndarray]]):
    x0 = get_initial_estimate(images, camera_matrix, dist_coeffs)
    def fun(x):
        return residual(x, x0, camera_matrix, dist_coeffs, images)[0]
    def jac(x):
        return residual(x, x0, camera_matrix, dist_coeffs, images)[1]
    res = scipy.optimize.least_squares(
        fun=fun,
        jac=jac,
        x0=x0,
        max_nfev=1000,
        verbose=2,
    )
    return res

In [37]:
observed_points = get_observed_points('./marker_calibration_pics/f30/*.jpg', arucoParams)
display(observed_points[0])

{6: array([[934.089  , 761.57025],
        [918.8917 , 754.8626 ],
        [916.9027 , 673.8126 ],
        [931.9539 , 684.1033 ]], dtype=float32),
 5: array([[890.6587, 753.3265],
        [810.4425, 758.5669],
        [809.3852, 677.4712],
        [888.0601, 671.9853]], dtype=float32),
 0: array([[829.3693 , 569.25806],
        [792.0881 , 584.74066],
        [791.75146, 510.6473 ],
        [828.23914, 492.6829 ]], dtype=float32),
 1: array([[927.2496 , 573.3005 ],
        [860.36365, 566.3303 ],
        [859.00037, 489.39542],
        [924.7339 , 498.5793 ]], dtype=float32)}

In [5]:
from marker_tracker import readCameraParameters


camera_matrix, dist_coeffs = readCameraParameters("./camera_params_c922_f30.yml")
x0 = get_initial_estimate(observed_points, camera_matrix, dist_coeffs)
res, jac = residual(x0, camera_matrix, dist_coeffs, observed_points)

In [9]:
display(np.sum(np.square(res))*0.5)
display(np.min(res))

5257.77734375

-14.997314

In [38]:
result = calibrate_markers(camera_matrix, dist_coeffs, observed_points)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.6784e+03                                    6.03e+04    
       1              6         3.8184e+03      8.60e+02       4.10e-03       8.79e+04    
       2              8         3.5488e+03      2.70e+02       2.05e-03       4.88e+04    
       3             10         3.5321e+03      1.66e+01       1.03e-03       1.83e+04    
       4             11         3.5046e+03      2.75e+01       1.03e-03       2.25e+04    
       5             12         3.4913e+03      1.33e+01       2.05e-03       3.35e+04    
       6             13         3.4707e+03      2.06e+01       2.05e-03       3.29e+04    
       7             14         3.4609e+03      9.75e+00       2.05e-03       3.36e+04    
       8             15         3.4219e+03      3.90e+01       5.13e-04       7.03e+03    
       9             16         3.4195e+03      2.49e+00       1.03e-03       1.24e+04    

In [39]:
import pickle
markersOnPenCalibrated = {}
posesCalibrated = result.x[0:8*4*3].reshape((8, 4, 3))
for i in range(8):
    markersOnPenCalibrated[i+92] = posesCalibrated[i,:,:]
with open("../markersOnPenCalibrated.pkl", "wb") as f:
    pickle.dump(markersOnPenCalibrated, f)

In [16]:
compare = np.column_stack((x0, result.x))